# Baseline Model Implementation using Lifelines package

In [1]:
import pandas as pd
import numpy as np
import os
from lifelines import KaplanMeierFitter,CoxPHFitter

In [2]:
train_path = '../files_provided/data/train/'
test_path = '../files_provided/data/test/'

In [61]:
features_to_select = ['original_shape_Sphericity','original_shape_SurfaceVolumeRatio','original_shape_Maximum3DDiameter',
                      'SourceDataset','Nstage','original_glcm_JointEntropy',
                      'original_glcm_Idn','original_glcm_Idmn']

In [62]:
def get_features_from_csvs(path):
    """Extract features from radiomics and clinical_data csvs into a combined DataFrame"""
    dfr = pd.read_csv(path + 'features/radiomics.csv',skiprows=[0,2])
    dfc = pd.read_csv(path + 'features/clinical_data.csv')
    # Labeling the unlabled column
    dfr.columns.values[0] = "PatientID"
    # Changing the 'SourceDataset' column to categorical
    dfc.SourceDataset = pd.Categorical(dfc.SourceDataset)
    dfc.SourceDataset = dfc.SourceDataset.cat.codes
    # Joining the dataframes on 'PatientID'
    df_joined = dfc.join(dfr.set_index('PatientID'), on='PatientID')
    return df_joined

# print(df_joined.head())

In [63]:
# Creating the Training DataFrame
df_train = get_features_from_csvs(train_path)
dfl = pd.read_csv(train_path + 'train_labels.csv')
# Join with labels 
df_train = df_train.join(dfl.set_index('PatientID'), on='PatientID')
# Selecting the 8 baseline features along with ['SurvivalTime','Event']
df_train = df_train[features_to_select + ['SurvivalTime','Event']]
# df_train

In [64]:
# Creating the Testing DataFrame
df_test = get_features_from_csvs(test_path)
# Selecting the 8 baseline features
df_test = df_test[features_to_select]
# df_test

In [65]:
# Using the Cox’s proportional hazard model implementation from the 'lifelines' package
cph = CoxPHFitter()
cph.fit(df_train, 'SurvivalTime', event_col='Event')
cph.print_summary()

In [66]:
# Get test PatientsID list
df = pd.read_csv(test_path + 'features/clinical_data.csv')
test_patient_list = df['PatientID'].to_list()  
len(test_patient_list)

125

In [67]:
# Predicting on the test set
survival_times = cph.predict_expectation(df_test)
survival_times = list(survival_times.values.flatten())
df_predicted = pd.DataFrame({'PatientID':test_patient_list, 'SurvivalTime':survival_times, 'Event':np.nan})
print(df_predicted)
df_predicted.to_csv('out.csv',index=False)

     PatientID  SurvivalTime  Event
0           13    652.803074    NaN
1          155   1444.081720    NaN
2          404    828.677308    NaN
3          407    965.317241    NaN
4            9   2764.311832    NaN
..         ...           ...    ...
120         66    365.062549    NaN
121        132    671.351587    NaN
122        169   2455.078042    NaN
123        199    421.190052    NaN
124        274    853.650339    NaN

[125 rows x 3 columns]
